In [1]:
# ================================================
# Clasificación de Calidad de Señal con MLP 
# ================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [2]:
# Cargar datos
df = pd.read_csv("Dataset_transmision_act.csv")

In [3]:
# Selección de columnas relevantes

df = df[["RSRP_dBm", "SINR_dB", "CalidadSenal"]]

In [4]:
#  Convertir etiquetas a solo 2 clases
#    - "Regular" se agrupa como "Mala"

df["CalidadSenal"] = df["CalidadSenal"].replace({"Regular": "Mala"})


In [5]:
# Convertir etiquetas a solo 2 clases
#   "Regular" se agrupa como "Mala"

df["CalidadSenal"] = df["CalidadSenal"].replace({"Regular": "Mala"})

# 4. Filtrar solo filas con Buena o Mala
df = df[df["CalidadSenal"].isin(["Buena", "Mala"])]

In [6]:
# Variables predictoras y target

X = df[["RSRP_dBm", "SINR_dB"]].values
y = df["CalidadSenal"].values

# Codificar target: 0 = Mala, 1 = Buena
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [7]:
# Normalizar X
scaler = StandardScaler()
X = scaler.fit_transform(X)

#  Dividir en train y test

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [8]:
#  Convertir a tensores PyTorch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

print("Tamaño train:", len(train_dataset))
print("Tamaño test:", len(test_dataset))
print("Clases:", encoder.classes_)


Tamaño train: 4000
Tamaño test: 1000
Clases: ['Buena' 'Mala']


In [9]:
# Definir el MLP

class MLP(pl.LightningModule):
    def __init__(self, input_dim=2, hidden_dim=64, output_dim=2):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


In [10]:
#  Entrenamiento

model = MLP(input_dim=2, hidden_dim=64, output_dim=2)

trainer = pl.Trainer(max_epochs=20, accelerator="cpu")  # usa "gpu" si tienes CUDA
trainer.fit(model, train_loader, test_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | layer1  | Linear           | 192    | train
1 | layer2  | Linear           | 4.2 K  | train
2 | layer3  | Linear           | 130    | train
3 | relu    | ReLU             | 0      | train
4 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
4.5 K     Trainable params
0         Non-trainable params
4.5 K     Total params
0.018     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\User\Desktop\UPSE\Proyecto-Mod-IA\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\User\Desktop\UPSE\Proyecto-Mod-IA\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [11]:
#  Evaluación final

trainer.validate(model, test_loader)


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9990000128746033     │
│         val_loss          │   0.0052378540858626366   │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.0052378540858626366, 'val_acc': 0.9990000128746033}]

In [12]:
# Evaluación con métricas

# Pasar el modelo a evaluación
model.eval()

# Obtener predicciones sobre el set de prueba
y_pred_list = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        y_logits = model(X_batch)
        y_pred = torch.argmax(y_logits, dim=1)
        y_pred_list.append(y_pred)

# Concatenar todos los lotes
y_pred = torch.cat(y_pred_list).cpu().numpy()
y_true = y_test_tensor.cpu().numpy()


In [13]:
# Métricas de desempeño
# ============================
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Recall (macro):", recall_score(y_true, y_pred, average="macro"))
print("Recall (weighted):", recall_score(y_true, y_pred, average="weighted"))
print("F1 (macro):", f1_score(y_true, y_pred, average="macro"))
print("F1 (weighted):", f1_score(y_true, y_pred, average="weighted"))

# Como son 2 clases, podemos calcular ROC-AUC binario
print("ROC-AUC:", roc_auc_score(y_true, y_pred))

Accuracy: 0.999
Recall (macro): 0.9994318181818183
Recall (weighted): 0.999
F1 (macro): 0.997641058787174
F1 (weighted): 0.9990017904363806
ROC-AUC: 0.9994318181818183


In [14]:
# Matriz de confusión

cm = confusion_matrix(y_true, y_pred)
print("\nMatriz de confusión:\n", cm)

# Reporte más detallado
print("\nReporte de clasificación:\n", classification_report(y_true, y_pred, target_names=encoder.classes_))



Matriz de confusión:
 [[120   0]
 [  1 879]]

Reporte de clasificación:
               precision    recall  f1-score   support

       Buena       0.99      1.00      1.00       120
        Mala       1.00      1.00      1.00       880

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

